## UPAC96 Capture

This notebook demostrates how to capture events with the UPAC96

### Naludaq Version
*Max Version*: `0.17.2`  
*Min Version*: `0.17.2`

In [ ]:
# Print Naludaq version
import naludaq
print(f"Naludaq version: {naludaq.__version__}")

### Compatible Boards
+ `UPAC96`
    + Firmware: `v911` - `v911`


In [ ]:
%load_ext autoreload
%autoreload 2

from collections import deque
from logging import getLogger, DEBUG, INFO, StreamHandler, Formatter

from naludaq.board import Board, startup_board
from naludaq.communication import AnalogRegisters, DigitalRegisters, ControlRegisters
from naludaq.daq import DebugDaq
from naludaq.parsers import Upac96Parser

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import serial
import time

### Logger

In [ ]:
def setup_logger(connection_level = DEBUG):
    """Setup a logger for Naludaq to print out debug information.
    
    Args:
        connection_level (int): Sets logging level for UART/FTDI connections.
            Warning: If set too low, VS Code may crash
    """
    logger = getLogger()
    logger.setLevel(DEBUG)

    # UART is very verbose, keep on info unless something is broken.
    # getLogger("naludaq.UART").setLevel(connection_level)
    # getLogger("naludaq.FTDI").setLevel(connection_level)

    handler = StreamHandler()
    handler.setFormatter(Formatter("%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s"))
    logger.addHandler(handler)

    return logger

try:
    logger
except:
    logger = setup_logger()


## Board Connection

In [ ]:
board = Board('upac96')
board.load_registers()
board.get_uart_connection(serial_number='FTCKIDF5', baud=115200)


### Startup Routine

In [ ]:
startup_board(board)

In [ ]:
cr = ControlRegisters(board)
print(cr.read('identifier'))

In [ ]:
cr = ControlRegisters(board)
# Flush fifos, disable USB & enable UART
cr.write('usb_fifo_disable', 1)
cr.write('uart_fifo_disable', 1)
time.sleep(0.2)
cr.write('usb_fifo_disable', 1)
cr.write('uart_fifo_disable', 0)
time.sleep(0.2)

### Get Ready for Data

In [ ]:
dr = DigitalRegisters(board)
# dr.write('enable_testpattern', 0)
cr.write('continuousmode', 1)
cr.write('arm', 1)
time.sleep(0.25)
cr.write('arm', 0)

# "00"   Software Trigger (Default)
# "01"   UDC Self Trigger
# "10"   FPGA auto trigger 1 Hz
# "11"   External Trigger
cr.write('trigger_select', 0x00)

### Get Data

In [ ]:
parser = Upac96Parser(board.params)
ddaq = DebugDaq(board, store_raw_data=True, custom_parser = parser)

In [ ]:
ddaq.start_capture()
time.sleep(0.1)
board.connection.send('C0000003')
time.sleep(24) # 2 sec for 2M baud, 24 sec for 115200
ddaq.stop_capture()
print(f"Output Buffer len: {len(ddaq.output_buffer)}")

In [ ]:
board.disconnect()

## Graph Data

In [ ]:
# Setup graph style
cmap = matplotlib.cm.get_cmap('ocean')
matplotlib.rcParams.update({'font.size': 18})
matplotlib.rcParams.update({'font.family': 'monospace'})

In [ ]:
event = 1
data = ddaq.output_buffer[event]['data']
tim = ddaq.output_buffer[event]['time']

channels = data.shape[0]
fig = plt.figure(figsize=(8,8), constrained_layout=True)
plt.xlabel('Time')
plt.ylabel('ADC Counts')
plt.title('UPAC96 Capture')
for channel in [0]:
    ch_data = data[channel]
    ch_time = tim[channel]
    ax = plt.plot(ch_time, ch_data, '.', color=cmap(channel/channels))

plt.show()